In [ ]:
import os
import pandas as pd
from datetime import datetime
import json
import plotly.plotly as py
import matplotlib.pyplot as plt
import seaborn as sns
import random
import plotly.graph_objs as go
import plotly
%matplotlib inline

In [ ]:
DEBUG = True
MAX_LINES = 100
if not os.path.exists(os.path.expanduser("~/.plotly/.credentials")):
    plotly_user = input("plotly username: ")
    plotly_api_key = input("plotly api key: ")
    plotly.tools.set_credentials_file(username=plotly_user, api_key=plotly_api_key)

In [ ]:
log_input_file = input("csv log input file: ")
file = open(log_input_file, "r")

In [ ]:
operation_types = set()


def clean(tables):
    tables = tables.strip('`').strip('\'').strip()
    list_tables = tables.split(',')
    for i in range(len(list_tables)):
        list_tables[i] = remove_suffixes(list_tables[i], [' t0', ' t1', ' t2'])
        list_tables[i] = remove_prefixes(list_tables[i], ['mysql.'])
    list_tables = list(set(list_tables))
    return list_tables


def remove_suffixes(text, suffixes):
    for suffix in suffixes:
        if text.endswith(suffix):
            text = text[:-len(suffix)].strip()
    return text


def remove_prefixes(text, prefixes):
    for prefix in prefixes:
        if text.startswith(prefix):
            text = text[len(prefix):].strip()
    return text


def extract_tables_from_select(argument):
    argument = argument.lower()
    if 'from' in argument.lower():
        return argument.split(' from ')[1].split(' group by ')[0].split(' order by ')[0].split(' where ')[0]
    elif 'table_name = ' in argument:
        return argument.split('table_name = ')[1]
    elif 'from' in argument:
        return argument.split('from')[1].split('where')[0]
    else:
        if argument not in ['select 1', 'select @@session.tx_isolation',
                                     'select database()', 'select user()',
                                     'select @@global.sync_binlog']:
            if DEBUG:
                print("No tables found:", argument)
        return None

    
def load_from(file, max_lines):
    i = 0
    data = []
    
    for i, line in enumerate(file):
        parsed = line.split('\"')
        parsed = [parsed[i] for i in range(len(parsed)) if i % 2 != 0]

        if i == 0:
            fields = parsed
            print(fields)
        elif max_lines is not None and i > max_lines:
            break
        else:
            entry = {field: val for (field, val) in zip(fields, parsed)}
            entry['operation_type'] = entry['argument'].split(' ')[0].upper()
            operation_types.add(entry['operation_type'])
            entry[entry['operation_type']] = 1
            entry['tables'] = None

            if entry['operation_type'] == 'SELECT':
                entry['tables'] = extract_tables_from_select(entry['argument'])
            elif entry['operation_type'] == 'UPDATE':
                if 'SET' in entry['argument']:
                    entry['tables'] = entry['argument'].split('SET')[0].split('UPDATE')[1]
                else:
                    print(entry['argument'])
                    entry['tables'] = None
            elif entry['operation_type'] == 'INSERT':
                entry['tables'] = entry['argument'].split('INSERT INTO')[1].split('(')[0]
            elif entry['operation_type'] == 'SHOW' and 'TABLES FROM ' in entry['argument']:
                entry['tables'] = entry['argument'].split('TABLES FROM ')[1].split(' ')[0]
            else:
                if entry['argument'] != 'commit':
                    continue

            if entry['tables'] is not None:
                entry['table_list'] = clean(entry['tables'])
                for table in entry['table_list']:
                    entry[table] = 1
                entry['tables'] = (', ').join(entry['table_list'])

            data.append(entry)

    file.close()
    return data

data = load_from(file, max_lines = MAX_LINES)

In [ ]:
df = pd.DataFrame(data)
df

In [ ]:
df['event_time'] = pd.to_datetime(df['event_time'])

In [ ]:
df['operation_type'].value_counts()

In [ ]:
df['tables'].value_counts()

In [ ]:
timeseries = df.copy()
timeseries.index = timeseries['event_time']
del timeseries['event_time']
timeseries.columns

In [ ]:
timeseries.columns

In [ ]:
def generate_timeseries_data(cols, colors=None):
    if colors == None:
        colors = ["#{:06x}".format(random.randint(0, 0xFFFFFF)) for i in range(len(cols))]
    
    data = []
    
    for col, color in zip(cols, colors):
        series = timeseries[[col]].groupby(level=0).count().resample('5T').sum()
        data.append(go.Scatter(x=series.index,
                             y=series[col],
                             name=col,
                             line = dict(color = color),
                             opacity = 0.8))
    return data


data = generate_timeseries_data([field for field in timeseries.columns if field in operation_types])

layout = dict(
    title='Time Series by Operation Type',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=30,
                     label='30m',
                     step='minute',
                     stepmode='backward'),
                dict(count=1,
                     label='1h',
                     step='hour',
                     stepmode='backward'),
                dict(count=4,
                     label='4h',
                     step='hour',
                     stepmode='backward'),
                dict(count=6,
                     label='6h',
                     step='hour',
                     stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(),
        type='date'
    )
)

fig = dict(data=data, layout=layout)
py.iplot(fig)

In [ ]:
tbls = [field for field in timeseries.columns if field in ['dce_annotation_user_info', 'dce_aws_s3_archive_mapping', 'dce_metadata_synchronize',
       'dce_metadata_synchronize_wf', 'dce_otherseries','dce_transcript_job_control', 'general_log limit 10',
       'information_schema.innodb_sys_tables', 'information_schema.tables',
       'matterhorn', 'mh_annotation', 'mh_capture_agent_state',
       'mh_episode_asset', 'mh_episode_episode', 'mh_episode_version_claim',
       'mh_group', 'mh_group_member', 'mh_host_registration', 'mh_incident',
       'mh_job', 'mh_job_argument', 'mh_organization', 'mh_organization_node',
       'mh_organization_property', 'mh_search', 'mh_service_registration',
       'mh_user', 'mh_user_action', 'mh_user_ref', 'mh_user_session',
       'rds_configuration', 'rds_heartbeat2', 'rds_history',
       'rds_replication_status', 'sequence']]

py.iplot(dict(data=generate_timeseries_data(tbls), layout=layout))

In [ ]:
# REGULAR Matplotlib chart
sns.set_context("poster")

plt.figure(figsize=(20,10))
operations = ['SELECT', 'UPDATE', 'INSERT', 'SHOW']
for operation in operations:
    plt.plot(timeseries[operation].groupby(level=0).count(), label=operation)
plt.ylabel("occurances")
plt.legend();

In [ ]:
# ## Export data to json
# df_json = df.copy().drop(columns=['table_list'])
# data = df_json.to_json(orient='records')

# with open('output.json', 'w') as outfile:
#     json.dump(data, outfile)